In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline

import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Case study - zbiór `Boston House Dataset`

In [ ]:
from sklearn import datasets
boston = datasets.load_boston()

In [ ]:
print(boston['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

## Model prostej regresji liniowej
1. Podziel dane na część treningową i na część testową w stosunku 7:3.


2. Interesuje nas zależność wielkości ceny domu od średniej liczby pokoi w domu (zmienne `PRICE` i `RM`). Określ zmienną zależną i niezależną, a następnie:

      a) Narysuj wykres rozproszenia tych zmiennych. 
      
      b) Czy zmienne te są zależne? 
      
      c) Oblicz dla nich współczynnik korelacji Pearsona. 


3. Wyznacz funkcję regresji wielkości ceny domu względem średniej liczby pokoi. Narysuj ją na wykresie rozproszenia.


4. Zweryfikuj dopasowanie modelu:

    a) Czy istnieje związek pomiędzy zmienną zależną a niezależną?
    
    b) Czy wyraz wolny jest istotny w modelu?
    
    c) Oblicz i zinterpretuj wartość współczynnika determinacji $R^2$.
    
    d) Sprawdź, czy spełnione jest założenie o rozkładzie normalnym reszt.
    
    e) Zweryfikuj dopasowanie modelu, sporządź wykresy diagnostyczne: wykres wartości przewidywanych względem prawdziwych, wykres residuów względem wartości dopasowanych.
    
    f) Wyznacz i porównaj jakość dopasowania i predykcji za pomocą różnych miar (np. MSE, MAE, $R^2$).

## Model wielorakiej regresji liniowej

5. Wyznacz model liniowy najlepiej opisujący zależność zależność wielkości ceny domu od pozostałych dostępnych zmiennych:

    a) Czy istnieje związek pomiędzy zmienną zależną a którąkolwiek ze zmiennych niezależnych?
    
    b) Dla których zmiennych niezależnych Xj, j=1,...,10 można odrzucić hipotezę zerową  H0: βj = 0?
    
    c) Biorąc pod uwagę odpowiedź na poprzednie pytanie, wyznacz mniejszy model w oparciu tylko o zmienne, które faktycznie mają istotny wpływ na wielkość sprzedaży.
    
    d) Porównaj dopasowanie obu modeli (większego i mniejszego), sporządź wykresy diagnostyczne.
    
    e) Wyznacz i porównaj jakość dopasowania i predykcji dla modelu mniejszego i większego.

## Regularyzacja

6. Zastosuj regularyzację

    a) grzbietową
    
    b) Lasso


## Model regresji wielomianowej

7. Dopasuj model regresji wielomianowej:
        
    a) Znajdź optymalny stopień wielomianu.
    
    b) Zweryfikuj dopasowanie modelu, sporządź wykresy diagnostyczne.
    
    c) Zastosuj regularyzację znajdując najlepszy parametr $\alpha$.

## Podsumowanie

8. Dokonaj porównania dopasowanych powyżej modeli regresji pod względem błędu dopasowania i błędu predykcji (dane końcowe przedstaw za pomocą tabeli). 
    
**Pamiętaj o graficznej ewaluacji modeli regresji!!!**

In [ ]:
boston_full = pd.DataFrame(boston['data'], columns=boston['feature_names'])
boston_full['PRICE'] = boston['target']
print(boston_full)

        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0    0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1    0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2    0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3    0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4    0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...    ...   
501  0.06263   0.0  11.93   0.0  0.573  6.593  69.1  2.4786  1.0  273.0   
502  0.04527   0.0  11.93   0.0  0.573  6.120  76.7  2.2875  1.0  273.0   
503  0.06076   0.0  11.93   0.0  0.573  6.976  91.0  2.1675  1.0  273.0   
504  0.10959   0.0  11.93   0.0  0.573  6.794  89.3  2.3889  1.0  273.0   
505  0.04741   0.0  11.93   0.0  0.573  6.030  80.8  2.5050  1.0  273.0   

     PTRATIO       B  LSTAT  PRICE  
0       15.3  396.90   4.98   24.0  
1       17.8  396.90   9.

In [ ]:
X, y = pd.DataFrame(boston['data'], columns=boston['feature_names']), boston['target']
y

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 18.7, 16. , 22.2, 25. , 33. , 23.5,
       19.4, 22. , 17.4, 20.9, 24.2, 21.7, 22.8, 23.4, 24.1, 21.4, 20. ,
       20.8, 21.2, 20.3, 28. , 23.9, 24.8, 22.9, 23.9, 26.6, 22.5, 22.2,
       23.6, 28.7, 22.6, 22. , 22.9, 25. , 20.6, 28.4, 21.4, 38.7, 43.8,
       33.2, 27.5, 26.5, 18.6, 19.3, 20.1, 19.5, 19.5, 20.4, 19.8, 19.4,
       21.7, 22.8, 18.8, 18.7, 18.5, 18.3, 21.2, 19.2, 20.4, 19.3, 22. ,
       20.3, 20.5, 17.3, 18.8, 21.4, 15.7, 16.2, 18. , 14.3, 19.2, 19.6,
       23. , 18.4, 15.6, 18.1, 17.4, 17.1, 13.3, 17.8, 14. , 14.4, 13.4,
       15.6, 11.8, 13.8, 15.6, 14.6, 17.8, 15.4, 21

#Model prostej regresji liniowej

## Model wielorakiej regresji liniowej

#Regularyzacja

#Regresja wielomianowa

### Wyniki
